# <img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/header.png?raw=1" width="1000"/>


# Mapeo de inundaciones mediante imágenes de radar Sentinel-1

<img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/example.png?raw=1" width="1000"/>

***

El objetivo de esta práctica recomendada The objective of this [práctica recomendada](https://un-spider.org/advisory-support/recommended-practices) es determinar la extensión de las áreas inundadas. Mediante el uso de imágenes satelitales de radar de apertura sintética (SAR) para el mapeo de la extensión de las inundaciones. Esta práctica, constituye una solución viable para el procesamiento rápido de imágenes Sentinel-1, ya que proporciona información de inundaciones casi en tiempo real a las agencias de ayuda para apoyar la acción humanitaria. La alta confiabilidad de los datos, así como la ausencia de restricciones geográficas y la accesibilidad a las zonas afectadas, enfatizan el potencial de esta tecnología.

Este cuaderno de Jupyter está optimizado para su uso con Google Colab. Al ser un entorno basado en la computación en la nube, aprovecha los recursos técnicos externos y, por lo tanto, permite que esta herramienta pueda funcionar en dispositivos con potencia informática limitada, como teléfonos y tabletas, en áreas con escaso ancho de banda. 
Este cuaderno de Jupyter Notebook cubre toda la cadena de procesamiento, desde la consulta de datos, descarga, hasta la exportación de un producto final de máscara de inundación mediante el uso de imágenes SAR de libre acceso de Sentinel-1. El flujo de trabajo de la herramienta sigue la práctica recomendada de ONU-SPIDER sobre [mapeo de inundaciones basado en radar](https://un-spider.org/advisory-support/recommended-practices/recommended-practice-radar-based-flood-mapping), como se ilustra en la siguiente figura. Después de ingresar las especificaciones del usuario, los datos de Sentinel-1 se pueden descargar directamente desde del <a href="https://scihub.copernicus.eu/"> Copernicus Open Access Hub </a>. Posteriormente, los datos se procesan y almacenan en una variedad de formatos de salida.

<img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/charts/chart0.png?raw=1" width="1000"/>

***

***Estructura del archivo***  
El Jupyter Notebook crea una carpeta llamada *'mapeo-de-extensión-de-inundación'* en Google Drive. Las imágenes de Sentinel-1 deben almacenarse en una subcarpeta llamada *'entrada'*. Si no se proporciona ninguna imagen, la subcarpeta se creará automáticamente al acceder a la herramienta y descargar los datos del <a href="https://scihub.copernicus.eu/"> Copernicus Open Access Hub </a>. Si posee un área de interés (AOI) (formatos compatibles: GeoJSON, SHP, KML, KMZ), debe colocarse en una subcarpeta llamada *'AOI'*. Si no hay ninguno archivo disponible, una herramienta le permitirá dibujar manualmente y/o cargar archivos de AOI almacenados localmente. Por razones de selección automática de archivos, se recomienda colocar solo un solo archivo AOI en la carpeta correspondiente. Sin embargo, si existen varios archivos, los archivos GeoJSON tienen prioridad, seguidos de SHP, KML y KMZ. Los datos procesados se almacenan en una subcarpeta llamada *'salida'*.  
Para ejecutar la herramienta sin interacción del usuario, todas las entradas deben estar claramente definidas. Esto significa que la subcarpeta *'entrada'* debe incluir una sola imagen Sentinel-1 y la subcarpeta *'AOI'* un solo archivo AOI. Todos los demás escenarios requieren interacción manual, como descargar datos o definir una AOI.

***Limitaticiones***  
Existen limitaciones para detectar vegetación inundada e inundaciones en áreas urbanas debido a la retrodispersión de doble rebote. Si las zonas inundadas y no inundadas se distribuyen de manera muy desigual en la imagen, es posible que el histograma no tenga un mínimo local claramente definido, lo que da lugar a resultados incorrectos en el proceso de binarización automática.

***

## Inicialización

El Jupyter Notebook aprovecha la API <a href="https://step.esa.int/docs/v6.0/apidoc/engine/"> ESA SNAP</a> mediante la interfaz de SNAP-Python <i > ágil </i>. El procedimiento de instalación y configuración se incluye en el paso de inicialización de esta herramienta, esto puede tardar unos minutos durante la ejecución inicial de este Jupyter Notebook.

In [ ]:
#@title <font color=#1B7192> Click para iniciar </font>  { display-mode: "form" }

#####################################################
################### CONFIGURACIÓN ###################
#####################################################

# subir a Google Drive
import os                                     # acceso a datos
import google.colab                           # Google Colab
import time                                   # # tiempo de la evaluación
import sys
if not os.path.isdir('/content/drive'):
    google.colab.drive.mount('/content/drive')

try:
    import snappy                             # SNAP Python interface
    import jpy                                # Python-Java bridge
except:
    with google.colab.output.use_tags('snappy'):
        sys.stdout.write('\nPreparación del entorno conda...\n')
        sys.stdout.flush()
        !pip install -q condacolab &> /dev/null
        import condacolab
        condacolab.install_miniconda()
        sys.stdout.write('\nInstalación rápida de la interfaz SNAP-Python. Esto puede tardar unos minutos.\n')
        sys.stdout.flush()
        !conda install -c terradue -c conda-forge snap=8.0.0 &> /dev/null
    google.colab.output.clear(output_tags='snappy')
try:
    import geopandas                          # manipulación y análisis de datos
except:
    with google.colab.output.use_tags('geopandas'):
        sys.stdout.write('\nInstalación del paquete geopandas....\n')
        sys.stdout.flush()
        !pip install geopandas &> /dev/null
    google.colab.output.clear(output_tags='geopandas')
try:
    from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt  # interface to Open Access Hub
except:
    with google.colab.output.use_tags('sentinelsat'):
        sys.stdout.write('\nInstalación del paquete sentinelsat....\n')
        sys.stdout.flush()
        !pip install sentinelsat &> /dev/null
    google.colab.output.clear(output_tags='sentinelsat')
try:
    from ipyfilechooser import FileChooser    # widget selector de archivos
except:
    with google.colab.output.use_tags('ipyfilechooser'):
        sys.stdout.write('\nInstalación del paquete ipyfilechooser....\n')
        sys.stdout.flush()
        !pip install ipyfilechooser &> /dev/null
    google.colab.output.clear(output_tags='ipyfilechooser')

# actualización de estado
with google.colab.output.use_tags('initialization'):
    sys.stdout.write('\n¡Inicialización exitosa!')
    sys.stdout.flush()
time.sleep(1)
google.colab.output.clear(output_tags='initialization')

## Entradas de usuario

<img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/charts/chart1.png?raw=1" width="1000"/>

Especifique en la celda del código a continuación: **i)** El tipo polarización que se procesará, **ii)** si los datos se descargarán del <a href="https://scihub.copernicus.eu/"> Copernicus Open Access Hub </a>, y **iii)** si los resultados intermedios deben graficarse durante el proceso. Esta sección también carga los módulos de Python relevantes para el siguiente análisis e inicializa las funcionalidades básicas.

In [ ]:
#@title <font color=#1B7192> Click para iniciar </font>  { display-mode: "form" }

####################################################
############### ENTRADAS DE USUARIO ################
####################################################

# tipo de polarizaciones a procesar
Polarisation  = 'VH'                    #@param ["VH", "VV", "ambas"]

DownloadImage = True                   #@param {type:"boolean"}

# mostrar resultados intermedios si se establece en 'true'
PlotResults   = True                    #@param {type:"boolean"}

#####################################################
###################### IMPORTAR #####################
#####################################################

# MODULO                                      # DESCRIPCIÓN
import sys
try:
    import snappy                             # SNAP Python interface
    import jpy                                # Python-Java bridge
    import geopandas                          # manipulación y análisis de datos
    from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt  # interface to Open Access Hub
    from ipyfilechooser import FileChooser    # widget selector de archivos
except:
    sys.exit('\nPor favor, ejecute primero la celda de inicialización anterior.')
import matplotlib.pyplot as plt               # crear visualizaciones
import numpy as np                            # computación cientifíca
import json                                   # codificador y decodificador GeoJSON
import glob                                   # acceso a datos
import os                                     # acceso a datos
import ipywidgets                             # controles de la UI
import time                                   # tiempo de la evaluación
import shutil                                 # operaciones de archivo
import google.colab                           # Google Colab
import folium                                 # visualización
from folium import plugins                    # visualización
from folium.plugins import MiniMap, Draw, Search # visualización
import skimage.filters                        # cálculo del umbral
import functools                              # funciones y operaciones de orden superior
from datetime import date                     # fecha, hora e intervalos
from IPython.display import display           # visualización
from osgeo import ogr, gdal, osr              # conversión de datos
from zipfile import ZipFile                   # gestión de archivos




####################################################
############# DEFINICIÓN DE FUNSIONES ##############
####################################################

def getAOI(path):
    try:
        file = readJSONFromAOI(path)
    except:
        print('No se encontró ningún archivo AOI. Dibuje y descargue el área de interés haciendo clic en el botón -Export- dentro del')
        print('mapa o cargue directamente un archivo AOI almacenado localmente usando el cuadro de diálogo debajo del mapa.\n')
        # crea el mapa
        f = folium.Figure(height=500)
        m = folium.Map(location=[0, 0], zoom_start=2.5, control_scale=True).add_to(f)
        # agregar el mapa base personalizado
        basemaps['Google Satellite Hybrid'].add_to(m)
        # agregar un panel de control de capa al mapa
        m.add_child(folium.LayerControl())
        # agregar minimapa
        m.add_child(MiniMap(tile_layer=basemaps['Google Satellite'], position='bottomright'))
        # agregar control de dibujo
        draw = Draw(export=True, filename='AOI_manual_%s.geojson' % str(date.today()), draw_options={'polyline': False, 'circle': False, 'marker': False, 'circlemarker': False})
        draw.add_to(m)
        # mostrar el mapa
        updater = display(f, display_id='m')
        print('\n')
        # cargar la sección 
        os.chdir('/content')
        uploaded = google.colab.files.upload()
        for fn in uploaded.keys():
            # copiar el archivo cargado a la carpeta GDrive
            aoi_path = os.path.join(directory, 'AOI')
            if not os.path.isdir(aoi_path):
                os.mkdir(aoi_path)
            shutil.copy2('/content/%s' % fn, aoi_path)
            # elimina el archivo original
            os.remove('/content/%s' % fn)
            file_path = '%s/%s' % (aoi_path, fn)
        file = readJSONFromAOI(aoi_path)
    
    return file



# La función busca el archivo del AOI, si no esta en formato GeoJSON se convierte y devuelve la ruta
def readJSONFromAOI(path):
    # busca el archivo GeoJSON en la subcarpeta 'AOI'
    if len(glob.glob('%s/*.geojson' % path)) == 1:
        file = glob.glob('%s/*.geojson' % path)[0]
    elif len(glob.glob('%s/*.json' % path)) == 1:
        file = glob.glob('%s/*.json' % path)[0]

    # convierte el SHP a GeoJSON si no es proporcionado el JSON
    elif len(glob.glob('%s/*.shp' % path)) == 1:
        file_name = os.path.splitext(glob.glob('%s/*.shp' % path)[0])[0].split('/')[-1]
        shp_file = geopandas.read_file(glob.glob('%s/*.shp' % path)[0])
        shp_file.to_file('%s/%s.json' % (path, file_name), driver='GeoJSON')
        file = glob.glob('%s/*.json' % path)[0]

    # convierte el KML a GeoJSON si no es proporcionado el JSON 
    elif len(glob.glob('%s/*.kml' % path)) == 1:
        file_name = os.path.splitext(glob.glob('%s/*.kml' % path)[0])[0].split('/')[-1]
        kml_file = gdal.OpenEx(glob.glob('%s/*.kml' % path)[0])
        ds = gdal.VectorTranslate('%s/%s.json' % (path, file_name), kml_file, format='GeoJSON')
        del ds
        file = glob.glob('%s/*.json' % path)[0]

    # convierte el KMZ a JSON si no se proporciona un JSON, SHP o KML
    elif len(glob.glob('%s/*.kmz' % path)) == 1:
        # open KMZ file and extract data
        with ZipFile(glob.glob('%s/*.kmz' % path)[0], 'r') as kmz:
            folder = os.path.splitext(glob.glob('%s/*.kmz' % path)[0])[0]
            kmz.extractall(folder)
        # convierte el KML a GeoJSON si la carpeta extraída contiene un archivo KML
        if len(glob.glob('%s/*.kml' % folder)) == 1:
            kml_file = gdal.OpenEx(glob.glob('%s/*.kml' % folder)[0])
            ds = gdal.VectorTranslate('%s/%s.json' % (path, folder.split('/')[-1]), kml_file, format='GeoJSON')
            del ds
            file = glob.glob('%s/*.json' % path)[0]
            # elimina el archivo comprimido KMZ
            shutil.rmtree(folder)
    # Permite cargar archivos AOI o dibujar una AOI manualmente si no se encontra ningún archivo
    else:
        raise FileNotFoundError

    return file


# gráfica el histograma de la banda de entrada y el umbral
# SNAP API: https://step.esa.int/docs/v6.0/apidoc/engine/
def plotBand(band, threshold, binary=False):
    # realce de color
    vmin, vmax = 0, 1
    # lee los valores de los píxeles
    w = band.getRasterWidth()
    h = band.getRasterHeight()
    band_data = np.zeros(w * h, np.float32)
    band.readPixels(0, 0, w, h, band_data)
    band_data.shape = h, w
    # realce de color
    if binary:
        cmap = plt.get_cmap('binary')
    else:
        vmin = np.percentile(band_data, 2.5)
        vmax = np.percentile(band_data, 97.5)
        cmap = plt.get_cmap('gray')
    # gráfica la banda
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))
    ax1.imshow(band_data, cmap=cmap, vmin=vmin, vmax=vmax)
    ax1.set_title(band.getName())
    # gráfica el histograma
    band_data.shape = h * w 
    ax2.hist(np.asarray(band_data[band_data != 0], dtype='float'), bins=2048)
    ax2.axvline(x=threshold, color='r')
    ax2.set_title('Histogram: %s' % band.getName())
    
    for ax in fig.get_axes():
        ax.label_outer()




####################################################
###################### CÓDIGO ######################
####################################################   
        
# selecciona el directorio de trabajo
directory = '/content/drive/MyDrive/mapeo-de-extensión-de-inundación'
if not os.path.isdir(directory):
    os.mkdir(directory)

# Agrega un mapa base personalizado a folium
basemaps = {
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True,
        show = False
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True,
        show = False
    )
}

## Descargar imagen

<img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/charts/chart2.png?raw=1" width="1000"/>

Esta sección permite el acceso y la descarga de datos interactivos desde el <a href="https://scihub.copernicus.eu/"> Copernicus Open Access Hub </a>. Requiere los respectivos datos de inicio de sesión y el período de detección deseado. Si se proporciona un archivo de área de interés AOI en la subcarpeta *'AOI'*, la herramienta busca y muestra las imágenes Sentinel-1 disponibles. Si no se proporciona un archivo AOI, un mapa interactivo permite dibujar y descargar el área de interés haciendo clic en el botón *'Exportar'* - dentro del mapa o cargar directamente un archivo AOI almacenado localmente. Al pasar el cursor sobre una imagen de Sentinel-1, se muestran el índice de mosaico y las fechas de ingestión. La siguiente tabla resume la información sobre todos los mosaicos disponibles y permite la descarga. Los datos se almacenan en la subcarpeta *'entrada'* creada automáticamente. El Open Access Hub proporciona un archivo en línea de al menos el último año de productos para su descarga inmediata. El acceso a productos anteriores que ya no están disponibles en línea activará automáticamente la recuperación de los archivos a largo plazo. La descarga real puede ser iniciada por el usuario una vez que se restauran los datos (dentro de las 24 horas).

In [ ]:
#@title <font color=#1B7192> Click para iniciar </font>  { display-mode: "form" }

####################################################
############### ENTRADAS DE USUARIO ################
####################################################

Username      = ''              #@param {type:"string"}
Password      = ''              #@param {type:"string"}
SensingPeriod_Start  = '2021-01-01'     #@param {type:"date"}
SensingPeriod_Stop   = '2021-01-08'     #@param {type:"date"}




####################################################
############# DEFINICIÓN DE FUNSIONES ##############
####################################################

# buscar y mostrar mosaicos Sentinel-1 disponibles
def queri(footprint):
    # print status
    with google.colab.output.use_tags('loading'):
        sys.stdout.write('\nCargando...')
        sys.stdout.flush()
    # buscar productos Copernicus del Open Access Hub con respecto a la zona de entrada y el período de detección
    period = (date(int(SensingPeriod_Start.split('-')[0]), int(SensingPeriod_Start.split('-')[1]), int(SensingPeriod_Start.split('-')[2])),
              date(int(SensingPeriod_Stop.split('-')[0]), int(SensingPeriod_Stop.split('-')[1]), int(SensingPeriod_Stop.split('-')[2])))
    try:
        products = api.query(footprint, date=period, platformname='Sentinel-1', producttype='GRD')
        print('Conectado con éxito al Copernicus Open Access Hub.\n', flush=True)
    except:
        sys.exit('\nLos datos de inicio de sesión no son válidos. Cambie el nombre de usuario y/o la contraseña.')
    # convertir el GeoJSON para graficar
    products_json = api.to_geojson(products)
    # generar una advertencia de que no hay imagen disponible en un período de detección dado
    if not products_json['features']:
        sys.exit('\nNo hay imágenes Sentinel-1 disponibles. Cambie el período de detección en la sección de entrada del usuario.')
    # convierte a dataframe para la visualización de tablas
    products_df = api.to_dataframe(products)
    # agrega un índice al dataframe
    indices = []
    for i in range (1, len(products_df.index)+1):
        indices.append('Tile %d' % i)
        products_json.features[i-1].properties['index'] = ' Tile %d' % i
    products_df.insert(0, 'index', indices, True) 

    # carga los productos para la visualización
    s1_tiles = folium.GeoJson(
        products_json,
        name='S1 tiles',
        show=True,
        style_function=lambda feature: {'fillColor': 'royalblue', 'fillOpacity' : 0.2},
        highlight_function=lambda x: {'fillOpacity' : 0.4},
        tooltip=folium.features.GeoJsonTooltip(
            fields=['index', 'beginposition'],
            aliases=['Índice:','Fecha:'],
        ),
    ).add_to(m)
    # agrega un mapa base personalizado
    basemaps['Google Satellite Hybrid'].add_to(m)
    # agrega un panel de control a la capa del mapa
    m.add_child(folium.LayerControl())
    # actualización del estado
    google.colab.output.clear(output_tags='loading')
    # update map
    updater.update(m)

    # imprime la tabla con botones de descarga
    grid = ipywidgets.GridspecLayout(len(products_df.index)+1, 5, height='250px')
    grid[0,0] = ipywidgets.HTML('<h3>Índice</h3>')
    grid[0,1] = ipywidgets.HTML('<h3>Fecha</h3>')
    grid[0,2] = ipywidgets.HTML('<h3>Polarización</h3>')
    grid[0,3] = ipywidgets.HTML('<h3>Tamaño</h3>')
    for i in range(len(products_df.index)):
        grid[i+1,0] = ipywidgets.Label(products_df['index'][i])
        grid[i+1,1] = ipywidgets.Label(str(products_df['beginposition'][i]))
        grid[i+1,2] = ipywidgets.Label(products_df['polarisationmode'][i])
        grid[i+1,3] = ipywidgets.Label(products_df['size'][i])
        grid[i+1,4] = ipywidgets.Button(description = 'Descargar')
        grid[i+1,4].on_click(functools.partial(on_downloadButton_clicked, tile_id=products_df.values[i][-1]))
    display(grid)

# descarga el mosaico elegido de Sentinel-1 en la subcarpeta 'entrada'
def on_downloadButton_clicked(b, tile_id):
    # obtener información del producto
    product_info = api.get_product_odata(tile_id)
    # comprobar si el producto está disponible
    if product_info['Online']:
        # compruebe si existe la carpeta de entrada, si no, cree la carpeta de entrada
        input_path = os.path.join(directory, 'entrada')
        if not os.path.isdir(input_path):
            os.mkdir(input_path)
        # cambiar a la subcarpeta 'entrada' para almacenar el producto
        os.chdir(input_path)
        # actualizar el estado
        print('\nEl producto  %s está en línea. Comenzar descarga.' % tile_id, flush=True)
        # descargue el producto
        api.download(tile_id)
        # volver al directorio de trabajo anterior
        os.chdir(directory)
    # mensaje de error cuando el producto no está disponible
    else:
        print('\nEl producto %s no está en línea. Debe solicitarse manualmente.\n' % tile_id, flush=True)



####################################################
###################### CÓDIGO ######################
####################################################

# comprobar la entrada del usuario si se solicita la descarga de la imagen
if DownloadImage:
    # cenectar a la API
    api = SentinelAPI(Username, Password, 'https://scihub.copernicus.eu/dhus')
    # obetener la ruta a la AOI
    file = getAOI('%s/AOI' % directory)
    # abrir el archivo del AOI GeoJSON y almacenar los datos
    with open(file, 'r') as f:
        data_json = json.load(f)
    # definir el centro del mapa según la estructura interna de GeoJSON
    try:
        # Formato GeoJSON si se proporciona KMZ
        center = [data_json['features'][0]['geometry']['coordinates'][0][0][0][1],
                  data_json['features'][0]['geometry']['coordinates'][0][0][0][0]]
    except:
        # Formato GeoJSON si se proporciona JSON o SHP
        center = [data_json['features'][0]['geometry']['coordinates'][0][0][1],
                  data_json['features'][0]['geometry']['coordinates'][0][0][0]]
    # crea mapa
    f = folium.Figure(height=500)
    m = folium.Map(location=center, zoom_start=6, control_scale=True).add_to(f)
    # agregar el AOI al mapa
    folium.GeoJson(file, name='AOI', style_function = lambda x: {'color':'green'}).add_to(m)
    footprint = geojson_to_wkt(data_json)
    updater = display(f, display_id='m')

    # buscar escenas Sentinel-1 disponibles
    queri(footprint)

## Procesamiento

<img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/charts/chart3.png?raw=1" width="1000"/>

Si existe más de una imagen de Sentinel-1 en la subcarpeta *'entrada'*, el usuario puede seleccionar cuál se utilizará para el procesamiento. El subconjunto se genera de acuerdo con el archivo AOI en la subcarpeta *'AOI'*. Si no se proporciona un archivo AOI, un mapa interactivo permite dibujar y descargar el área de interés haciendo clic en el botón *'Exportar'* - dentro del mapa o cargar directamente un archivo AOI almacenado localmente. Posteriormente, se realizan los siguientes pasos de procesamiento:

1. ***Aplicar el archivo de órbita***: El archivo de órbita proporciona información precisa sobre la posición y la velocidad del satélite. Con base en esta información, se actualizan los vectores de estado de la órbita en los metadatos del producto. Los archivos de órbita precisos están disponibles de días a semanas después de la generación del producto. Dado que este es un paso de procesamiento opcional, la herramienta continuará el flujo de trabajo en caso de que el archivo de órbita aún no esté disponible para permitir aplicaciones de mapeo rápido.


2. ***Eliminación de ruido térmico***: la corrección de ruido térmico se aplica a los productos Sentinel-1 Nivel-1 GRD que aún no se han corregido.


3. ***Calibración radiométrica***: El objetivo de esta calibración de SAR es proporcionar imágenes en las que los valores de los píxeles se puedan relacionar directamente con la retrodispersión de la escena de radar. Aunque las imágenes de SAR no calibradas son suficientes para un uso cualitativo, las imágenes de SAR calibradas son esenciales para el uso cuantitativo de los datos de SAR.


4. ***Filtrado de puntos***: las imágenes SAR tienen texturas inherentes llamadas puntos que degradan la calidad de la imagen y dificultan la interpretación de las características. Estos puntos son causados por la interferencia aleatoria constructiva y destructiva de las ondas de retorno desfasadas pero coherentes y retro-dispersadas por la dispersión elemental dentro de cada celda de resolución. La reducción de ruido de estos puntos se puede aplicar mediante filtrado espacial o procesamiento multilook. En este paso se utiliza un filtro de tipo Lee con un tamaño de ventana de 5 x 5.


5. ***Corrección del terreno***: Debido a las variaciones topográficas de una escena y la inclinación del sensor de satélite, las distancias en las imágenes SAR pueden distorsionarse. Los datos que no se dirijan directamente a la ubicación del nadir del sensor tendrán cierta distorsión. Por lo tanto, las correcciones del terreno están destinadas a compensar estas deformaciones para permitir una representación geométrica realista en la imagen.


6. ***Binarización***: para obtener una máscara de inundación binaria, se analiza el histograma para separar el agua de los píxeles que no son de agua. Debido a la geometría lateral de los sensores SAR y la superficie del agua comparativamente lisa, solo una proporción muy pequeña de retrodispersión se refleja en el sensor, lo que genera valores de píxeles comparativamente bajos en el histograma. El umbral utilizado para la separación se calcula automáticamente utilizando las implementaciones de <a href="https://scikit-image.org/"> scikit-image </a> y un uso combinado del <a href = "https: // doi .org / 10.1111 / j.1749-6632.1965.tb11715.x "> método mínimo </a> y el <a href =" https://www.semanticscholar.org/paper/A-threshold-selection-method-from- gray-level-Otsu / 1d4816c612e38dac86f2149af667a5581686cdef? p2df "> método de Otsu </a>. La capa <a href="http://due.esrin.esa.int/page_globcover.php"> GlobCover </a> de la Agencia Espacial Europea se utiliza para enmascarar los cuerpos de agua permanentes.


7. ***Filtrado de manchas***: En este paso se utiliza un filtro de mediana con un tamaño de ventana de 7 x 7.

In [ ]:
#@title <font color=#1B7192> Click para iniciar </font>  { display-mode: "form" }

####################################################
############# DEFINICIÓN DE FUNSIONES ##############
####################################################

def applySubset(path):
    # establecer la ruta correcta del archivo de entrada y crear el producto S1
    if len(files) is 1:
        file_path = path
    else:
        file_path = path.selected
    S1_source = snappy.ProductIO.readProduct(file_path)

    # leer las coordenadas geográficas de los metadatos de las imágenes Sentinel-1
    meta_data = S1_source.getMetadataRoot().getElement('Abstracted_Metadata')
    # define el centro del mapa de acuerdo con la imagen Sentinel-1
    S1_center = (meta_data.getAttributeDouble('centre_lat'), meta_data.getAttributeDouble('centre_lon'))
    # define el polígono que ilustra la imagen de Sentinel-1
    polygon_geom = {
      "type": "FeatureCollection",
      "features":
                [{"type": "Feature",
                "properties": {},
                "geometry": {"type": "Polygon", "coordinates": [[[meta_data.getAttributeDouble('first_near_long'), meta_data.getAttributeDouble('first_near_lat')],
                                                                [meta_data.getAttributeDouble('last_near_long'), meta_data.getAttributeDouble('last_near_lat')],
                                                                [meta_data.getAttributeDouble('last_far_long'), meta_data.getAttributeDouble('last_far_lat')],
                                                                [meta_data.getAttributeDouble('first_far_long'), meta_data.getAttributeDouble('first_far_lat')],
                                                                [meta_data.getAttributeDouble('first_near_long'), meta_data.getAttributeDouble('first_near_lat')]]]}}]}

    # obtener la ruta a las AOI
    file = getAOI('%s/AOI' % directory)
    # abrir archivo GeoJSON y almacenar los datos
    with open(file, 'r') as f:
        data_json = json.load(f)
    footprint = geojson_to_wkt(data_json)

    # crea el mapa
    f = folium.Figure(height=500)
    m = folium.Map(location = S1_center, zoom_start = 7.5, control_scale=True).add_to(f)
    # agregar el mosaico de S1 al mapa
    folium.GeoJson(polygon_geom, name='Sentinel-1 tile').add_to(m)
    # adiciona las AOI al mapa
    folium.GeoJson(file, name='AOI', style_function = lambda x: {'color':'green'}).add_to(m)
    # agregar un mapa base personalizado
    basemaps['Google Satellite Hybrid'].add_to(m)
    # agregar un panel de control de capa al mapa
    m.add_child(folium.LayerControl())
    # despliega el mapa
    updater = display(f, display_id='m')

    # Operador de subconjunto o recorte
    parameters = snappy.HashMap()
    parameters.put('copyMetadata', True)
    geom = snappy.WKTReader().read(footprint)
    parameters.put('geoRegion', geom)
    parameters.put('sourceBands', sourceBands)
    S1_crop = snappy.GPF.createProduct('Subset', parameters, S1_source)
    # actualiza el estado
    print('\nSubconjunto generado correctamente.\n', flush=True)

    # ejecutar el procesamiento
    processing(S1_crop)


# calcular y devolve el umbral de entrada para la 'Banda'
# SNAP API: https://step.esa.int/docs/v6.0/apidoc/engine/
def getThreshold(S1_band):
    # leer la banda
    w = S1_band.getRasterWidth()
    h = S1_band.getRasterHeight()
    band_data = np.zeros(w * h, np.float32)
    S1_band.readPixels(0, 0, w, h, band_data)
    band_data.shape = h * w
    # calcular el umbral utilizando el método Otsu
    threshold_otsu = skimage.filters.threshold_otsu(band_data)
    # calcular el umbral utilizando el método mínimo
    threshold_minimum = skimage.filters.threshold_minimum(band_data)
    # obtener el número de píxeles para ambos umbrales
    numPixOtsu = len(band_data[abs(band_data - threshold_otsu) < 0.1])
    numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])

    # si el número de píxeles en el umbral mínimo es inferior al 0,1% del número de píxeles en el umbral de Otsu
    if abs(numPixMinimum/numPixOtsu) < 0.001:
        # ajustar los datos de la banda de acuerdo a:
        if threshold_otsu < threshold_minimum:
            band_data = band_data[band_data < threshold_minimum]
            threshold_minimum = skimage.filters.threshold_minimum(band_data)
        else:
            band_data = band_data[band_data > threshold_minimum]
            threshold_minimum = skimage.filters.threshold_minimum(band_data)
        numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])
    # comprobar el umbral final
    if abs(numPixMinimum/numPixOtsu) < 0.001:
        threshold = threshold_otsu
    else:
        threshold = threshold_minimum

    return threshold


# calcula la máscara binaria o 'Producto' con respecto a la expresión en la matriz de cadenas
def binarization(S1_product, expressions):

    BandDescriptor = jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')
    targetBands = jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', len(expressions))

    # bucle a través de bandas
    for i in range(len(expressions)):
        targetBand = BandDescriptor()
        targetBand.name = '%s' % S1_product.getBandNames()[i]
        targetBand.type = 'float32'
        targetBand.expression = expressions[i]
        targetBands[i] = targetBand
    
    parameters = snappy.HashMap()
    parameters.put('targetBands', targetBands)    
    mask = snappy.GPF.createProduct('BandMaths', parameters, S1_product)

    return mask


# pasos de procesamiento
def processing(S1_crop):
    # aplica el archivo de órbita
    print('1. Aplica el archivo de orbita:          ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    # Continuar con el cálculo en caso de que aún no haya ningún archivo de órbita disponible.
    parameters.put('continueOnFail', True)
    S1_Orb = snappy.GPF.createProduct('Apply-Orbit-File', parameters, S1_crop)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # eliminación de ruido térmico
    print('2. Eliminación de ruido térmico:         ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('removeThermalNoise', True)
    S1_Thm = snappy.GPF.createProduct('ThermalNoiseRemoval', parameters, S1_Orb)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Operador de calibración
    print('3. Calibración radiométrica:             ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('outputSigmaBand', True)
    S1_Cal = snappy.GPF.createProduct('Calibration', parameters, S1_Thm)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Operador de filtro de manchas
    print('4. Filtrado de ruido (Speckle):          ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('filter', 'Lee')
    parameters.put('filterSizeX', 5)
    parameters.put('filterSizeY', 5)
    S1_Spk = snappy.GPF.createProduct('Speckle-Filter', parameters, S1_Cal)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Conversión del operador lineal a db
    S1_Spk_db = snappy.GPF.createProduct('LinearToFromdB', snappy.HashMap(), S1_Spk)

    # Operador de la corrección de terreno
    print('5. Corrección Topográfica:               ', end='', flush=True)
    parameters = snappy.HashMap()
    parameters.put('demName', 'SRTM 1Sec HGT')
    parameters.put('demResamplingMethod', 'BILINEAR_INTERPOLATION')
    parameters.put('imgResamplingMethod', 'NEAREST_NEIGHBOUR')
    parameters.put('pixelSpacingInMeter', 10.0)
    parameters.put('nodataValueAtSea', False)
    parameters.put('saveSelectedSourceBand', True)
    S1_TC = snappy.GPF.createProduct('Terrain-Correction', parameters, S1_Spk_db)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Binarización
    print('6. Binarización:                         ', end='', flush=True)
    start_time = time.time()
    # adiciona la capa del GlobCover 
    parameters = snappy.HashMap()
    parameters.put('landCoverNames', 'GlobCover')
    GlobCover = snappy.GPF.createProduct('AddLandCover', parameters, S1_TC)
    # matriz de cadena vacía para expresiones matemáticas de banda de binarización
    expressions = ['' for i in range(S1_TC.getNumBands())]
    # matriz vacía para umbral (s)
    thresholds = np.zeros(S1_TC.getNumBands())
    # bucle a través de las bandas
    for i in range(S1_TC.getNumBands()):
        # calcular el umbral de la banda y almacenarlo en una matriz flotante
        # utilice el producto S1_Spk_db por motivos de rendimiento. S1_TC provoca valores 0
        # que distorsionan el histograma y, por lo tanto, el resultado del umbral
        thresholds[i] = getThreshold(S1_Spk_db.getBandAt(i))
        # formular la expresión según el umbral y almacenarla en una matriz de cadenas
        expressions[i] = 'if (%s < %s && land_cover_GlobCover != 210) then 1 else NaN' % (S1_TC.getBandNames()[i], thresholds[i])
    # hacer binarización
    S1_floodMask = binarization(GlobCover, expressions)
    print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)

    # Operador de filtro de manchas o puntos
    print('7. Filtrado de ruido (Speckle):          ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('filter', 'Median')
    parameters.put('filterSizeX', 5)
    parameters.put('filterSizeY', 5)
    # definir la máscara de inundación como global para un acceso posterior
    global S1_floodMask_Spk
    S1_floodMask_Spk = snappy.GPF.createProduct('Speckle-Filter', parameters, S1_floodMask)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # salida
    if PlotResults:
        print('8. Graficar :                            ', end='', flush=True)
        start_time = time.time()
        for i in range(S1_TC.getNumBands()):
            plotBand(S1_TC.getBandAt(i), thresholds[i])
        print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)




####################################################
###################### CÓDIGO ######################
####################################################

# filtrar la(s) polarización(es) requerida(s) y establecer el nombre del archivo de salida en consecuencia
if Polarisation == 'ambas':
    sourceBands = 'Amplitude_VH,Intensity_VH,Amplitude_VV,Intensity_VV'
    output_extensions   = 'processed_VHVV'
elif Polarisation == 'VH':
    sourceBands = 'Amplitude_VH,Intensity_VH'
    output_extensions   = 'processed_VH'
elif Polarisation == 'VV':
    sourceBands = 'Amplitude_VV,Intensity_VV'
    output_extensions   = 'processed_VV'

# ruta del archivo de entrada .zip de Sentinel-1
input_path = os.path.join(directory, 'entrada')
# matriz de cadena vacía para almacenar archivos Sentinel-1 en la subcarpeta 'entrada'
files = []
# agregar archivos a la lista
for file in glob.glob1(input_path, '*.zip'):
    files.append(file)
# seleccione el archivo de entrada y comience a procesar si solo hay un archivo Sentinel-1 disponible
if len(files) == 1:
    input_name = files[0]
    print('Seleccionado:  %s\n' % input_name, flush=True)
    # aplicar subconjunto de acuerdo con los datos JSON
    applySubset('%s/%s' % (input_path, input_name))
# abrir la caja de diálogo para seleccionar el archivo de entrada si hay más o menos de uno disponible
else:
    print('Se ha encontrado más de un archivo Sentinel-1. Por favor seleccione.', flush=True)
    fc = FileChooser(input_path)
    fc.filter_pattern = '*.zip'
    fc.register_callback(applySubset)
    display(fc)

## Exportar Datos

<img src="https://github.com/UN-SPIDER/radar-based-flood-mapping-spanish/blob/main/resources/charts/chart4.png?raw=1" width="1000"/>

La máscara de inundación procesada se exporta como un GeoTIFF, SHP, KML y GeoJSON y se almacena en la subcarpeta *'salida'*. Un mapa interactivo muestra la máscara de inundación.

In [ ]:
#@title <font color=#1B7192> Click para iniciar </font>  { display-mode: "form" }

####################################################
####################### CODE #######################
####################################################

print('Exportando...\n', flush=True)
# compruebe si existen carpetas de salida, si no crea las carpetas
output_path = os.path.join(directory, 'salida')
if not os.path.isdir(output_path):
    os.mkdir(output_path)
GeoTIFF_path = os.path.join(output_path, 'GeoTIFF')
if not os.path.isdir(GeoTIFF_path):
    os.mkdir(GeoTIFF_path)
SHP_path = os.path.join(output_path, 'SHP')
if not os.path.isdir(SHP_path):
    os.mkdir(SHP_path)
KML_path = os.path.join(output_path, 'KML')
if not os.path.isdir(KML_path):
    os.mkdir(KML_path)
GeoJSON_path = os.path.join(output_path, 'GeoJSON')
if not os.path.isdir(GeoJSON_path):
    os.mkdir(GeoJSON_path)
# obtener el nombre del archivo si se utilizó el selector de archivos
if len(files) is not 1: input_name = fc.selected_filename

# escribir el archivo de salida como GeoTIFF
print('1. GeoTIFF:                   ', end='', flush=True)
start_time = time.time()
snappy.ProductIO.writeProduct(S1_floodMask_Spk, '%s/%s_%s' % (GeoTIFF_path, os.path.splitext(input_name)[0], output_extensions), 'GeoTIFF')
print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)

# convertir el GeoTIFF a SHP
print('2. SHP:                       ', end='', flush=True)
start_time = time.time()
# permitir que GDAL lance excepciones de Python
gdal.UseExceptions()
open_image = gdal.Open('%s/%s_%s.tif' % (GeoTIFF_path, os.path.splitext(input_name)[0], output_extensions))
srs = osr.SpatialReference()
srs.ImportFromWkt(open_image.GetProjectionRef())
shp_driver = ogr.GetDriverByName('ESRI Shapefile')
# matriz de cadena vacía para las bandas en GeoTIFF
output_shp = ['' for i in range(open_image.RasterCount)]
if open_image.RasterCount == 1:
    output_shp[0] = '%s/%s_processed_%s' % (SHP_path, os.path.splitext(input_name)[0], Polarisation)
else:
    VH_SHP_path = os.path.join(SHP_path, 'VH')
    if not os.path.isdir(VH_SHP_path):
        os.mkdir(VH_SHP_path)
    VV_SHP_path = os.path.join(SHP_path, 'VV')
    if not os.path.isdir(VV_SHP_path):
        os.mkdir(VV_SHP_path)
    output_shp[0] = '%s/%s_processed_VH' % (VH_SHP_path, os.path.splitext(input_name)[0])
    output_shp[1] = '%s/%s_processed_VV' % (VV_SHP_path, os.path.splitext(input_name)[0])
# bucles a través de las bandas en GeoTIFF
for i in range(open_image.RasterCount):
    input_band = open_image.GetRasterBand(i+1)
    output_shapefile = shp_driver.CreateDataSource(output_shp[i] + '.shp')
    new_shapefile = output_shapefile.CreateLayer(output_shp[i], srs=srs)
    new_shapefile.CreateField(ogr.FieldDefn('DN', ogr.OFTInteger))
    gdal.Polygonize(input_band, input_band.GetMaskBand(), new_shapefile, 0, [], callback=None)
    # filtra los atributos con valores distintos de 1 (debe ser NaN o el valor respectivo)
    new_shapefile.SetAttributeFilter('DN != 1')
    for feat in new_shapefile:
        new_shapefile.DeleteFeature(feat.GetFID())
    new_shapefile.SyncToDisk()
print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)

# convertir de SHP a KML
print('3. KML:                       ', end='', flush=True)
start_time = time.time()
if open_image.RasterCount == 1:
    shp_file = gdal.OpenEx('%s/%s_processed_%s.shp' % (SHP_path, os.path.splitext(input_name)[0], Polarisation))
    ds = gdal.VectorTranslate('%s/%s_processed_%s.kml' % (KML_path, os.path.splitext(input_name)[0], Polarisation), shp_file, format='KML')
    del ds
else:
    shp_file_VH = gdal.OpenEx('%s/%s_processed_VH.shp' % (VH_SHP_path, os.path.splitext(input_name)[0]))
    ds_VH = gdal.VectorTranslate('%s/%s_processed_VH.kml' % (KML_path, os.path.splitext(input_name)[0]), shp_file_VH, format='KML')
    del ds_VH
    shp_file_VV = gdal.OpenEx('%s/%s_processed_VV.shp' % (VV_SHP_path, os.path.splitext(input_name)[0]))
    ds_VV = gdal.VectorTranslate('%s/%s_processed_VV.kml' % (KML_path, os.path.splitext(input_name)[0]), shp_file_VV, format='KML')
    del ds_VV
print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)

# convertir de SHP a GeoJSON
print('4. GeoJSON:                   ', end='', flush=True)
start_time = time.time()
if open_image.RasterCount == 1:
    shp_file = geopandas.read_file('%s/%s_processed_%s.shp' % (SHP_path, os.path.splitext(input_name)[0], Polarisation))
    shp_file.to_file('%s/%s_processed_%s.json' % (GeoJSON_path, os.path.splitext(input_name)[0], Polarisation), driver='GeoJSON')
else:
    shp_file_VH = geopandas.read_file('%s/%s_processed_VH.shp' % (VH_SHP_path, os.path.splitext(input_name)[0]))
    shp_file_VH.to_file('%s/%s_processed_VH.json' % (GeoJSON_path, os.path.splitext(input_name)[0]), driver='GeoJSON')    
    shp_file_VV = geopandas.read_file('%s/%s_processed_VV.shp' % (VV_SHP_path, os.path.splitext(input_name)[0]))
    shp_file_VV.to_file('%s/%s_processed_VV.json' % (GeoJSON_path, os.path.splitext(input_name)[0]), driver='GeoJSON')
print('--- %.2f seconds ---\n' % (time.time() - start_time), flush=True)
print('Archivos almacenados correctamente en %s.\n' % output_path, flush=True)

# gráfica los resultados
meta_data = S1_floodMask_Spk.getMetadataRoot().getElement('Abstracted_Metadata')
S1_center = (meta_data.getAttributeDouble('centre_lat'), meta_data.getAttributeDouble('centre_lon'))
f = folium.Figure(height=500)
results_map = folium.Map(location = S1_center, zoom_start = 9, control_scale = True).add_to(f)
if open_image.RasterCount == 1:
    file = '%s/%s_processed_%s.json' % (GeoJSON_path, os.path.splitext(input_name)[0], Polarisation)
    folium.GeoJson(file, name='Flood Mask %s' % Polarisation, style_function = lambda x: {'color':'blue', 'opacity':'1', 'fillColor':'blue', 'fillOpacity':'1', 'weight':'0.8'}).add_to(results_map)
else:
    file_VV = '%s/%s_processed_VV.json' % (GeoJSON_path, os.path.splitext(input_name)[0])
    folium.GeoJson(file_VV, name='Flood Mask VV', style_function = lambda x: {'color':'red', 'opacity':'1', 'fillColor':'red', 'fillOpacity':'1', 'weight':'0.8'}).add_to(results_map)
    file_VH = '%s/%s_processed_VH.json' % (GeoJSON_path, os.path.splitext(input_name)[0])
    folium.GeoJson(file_VH, name='Flood Mask VH', style_function = lambda x: {'color':'blue', 'opacity':'1', 'fillColor':'blue', 'fillOpacity':'1', 'weight':'0.8'}).add_to(results_map)
# agregar un mapa base personalizado
basemaps['Google Satellite Hybrid'].add_to(results_map)
# agregar un panel de control de capa al mapa
results_map.add_child(folium.LayerControl(collapsed=False))
display(f)